# 🛠️ Part 4: Data Preparation - Handling Missing Values and Feature Engineering

**Goal:** To understand how to identify, evaluate the impact of, and strategically handle missing data. We will then learn fundamental **Feature Engineering** techniques to create new, predictive columns.

---
### Key Learning Objectives
1.  Quantify missing data using `.isnull().sum()` and percentages.
2.  Compare data loss when using different **dropping** strategies (`dropna()`).
3.  Implement different **imputation** (filling) strategies, including group-based filling.
4.  Create new features like `FamilySize`, `IsAlone`, and `Title` from existing columns.

In [5]:
import pandas as pd
import numpy as np
import os

# Load previously saved Titanic data or fallback to URL
try:
    titanic_df = pd.read_csv('data-visualization/data/titanic_snapshot.csv')
    print("✅ Loaded titanic_snapshot.csv")
except FileNotFoundError:
    url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
    titanic_df = pd.read_csv(url)
    print("📥 Loaded fresh Titanic data from web")

print(f"Dataset shape: {titanic_df.shape}")
print(f"Columns: {list(titanic_df.columns)}")

✅ Loaded titanic_snapshot.csv
Dataset shape: (891, 12)
Columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


## 1. Missing Data Analysis (The Problem)

Before any modeling, we must understand data completeness. Missing values can skew statistical results and cause machine learning models to fail.

The two main strategies are **Dropping** (removing rows/columns) and **Imputing** (filling in values).

In [6]:
print(f"\n=== MISSING DATA ANALYSIS ===")

# Count missing values per column
missing_counts = titanic_df.isnull().sum()
print("Missing values per column:")
print(missing_counts[missing_counts > 0])

# Missing data percentages
total_rows = len(titanic_df)
missing_percentages = (missing_counts / total_rows) * 100
print(f"\nMissing data percentages:")
for column, percentage in missing_percentages[missing_percentages > 0].items():
    print(f"  {column}: {percentage:.1f}%")

print(f"\n=== SAMPLE OF MISSING DATA ===")
age_missing = titanic_df[titanic_df['Age'].isnull()]
print(f"Sample passengers with missing age (showing 3 of {len(age_missing)}):")
print(age_missing[['Name', 'Sex', 'Age', 'Pclass', 'Survived']].head(3))


=== MISSING DATA ANALYSIS ===
Missing values per column:
Age         177
Cabin       687
Embarked      2
dtype: int64

Missing data percentages:
  Age: 19.9%
  Cabin: 77.1%
  Embarked: 0.2%

=== SAMPLE OF MISSING DATA ===
Sample passengers with missing age (showing 3 of 177):
                            Name     Sex  Age  Pclass  Survived
5               Moran, Mr. James    male  NaN       3         0
17  Williams, Mr. Charles Eugene    male  NaN       2         1
19       Masselmani, Mrs. Fatima  female  NaN       3         1


## 2. Handling Missing Data: Dropping

Dropping rows or columns is the simplest solution but risks significant **data loss**. We must assess the trade-off.

* **Dropping Rows:** Use `dropna(subset=[...])` to remove rows missing data in specific, important columns.
* **Dropping Columns:** If a column (like `Cabin`) is missing too much data (e.g., >70%), it might be better to remove the entire column (`df.drop(['Column'], axis=1)`).

In [7]:
# Drop rows with ANY missing value
complete_rows = titanic_df.dropna()
print(f"Complete rows (no missing values): {len(complete_rows)}")
print(f"Data loss if dropping all missing: {(len(titanic_df) - len(complete_rows)) / len(titanic_df) * 100:.1f}%")

# Drop the Cabin column if it's mostly empty (over 70% missing)
cabin_missing_pct = (titanic_df['Cabin'].isnull().sum() / len(titanic_df)) * 100
print(f"\nCabin column missing: {cabin_missing_pct:.1f}%")

df_no_cabin = titanic_df.drop(['Cabin'], axis=1)
print(f"Dropped Cabin column - now have {df_no_cabin.shape[1]} columns.")

Complete rows (no missing values): 183
Data loss if dropping all missing: 79.5%

Cabin column missing: 77.1%
Dropped Cabin column - now have 11 columns.


## 3. Handling Missing Data: Imputation (Filling)

Imputation fills missing values, allowing us to keep all rows. The choice of filling value is critical:

* **Simple Imputation:** Fill with the **median** (less sensitive to outliers than the mean) or **mode** (for categorical data).
* **Smart/Conditional Imputation:** Fill based on the median/mode of *similar* subgroups (e.g., filling a woman's age with the median age of all other women).

In [8]:
df_filled = titanic_df.copy()

# Simple Imputation for Age
median_age = df_filled['Age'].median()
df_filled['Age_filled_median'] = df_filled['Age'].fillna(median_age)
print(f"Filled Age missing values with overall median: {median_age:.1f}")

# Smart Imputation for Age (Group-based)
age_by_group = df_filled.groupby(['Sex', 'Pclass'])['Age'].median()
print("\nAge patterns by Sex and Class:")
print(age_by_group.round(1))

df_filled['Age_filled_smart'] = df_filled['Age']
for (sex, pclass), median_age in age_by_group.items():
    mask = (df_filled['Sex'] == sex) & (df_filled['Pclass'] == pclass) & (df_filled['Age'].isnull())
    df_filled.loc[mask, 'Age_filled_smart'] = median_age

# Imputation for Categorical (Embarked)
most_common_port = df_filled['Embarked'].mode()[0]
df_filled['Embarked_filled'] = df_filled['Embarked'].fillna(most_common_port)
print(f"\nFilled Embarked missing values with most common: '{most_common_port}'")

# Imputation for Cabin (Treating missing as a category)
df_filled['Cabin_filled'] = df_filled['Cabin'].fillna('Unknown')

Filled Age missing values with overall median: 28.0

Age patterns by Sex and Class:
Sex     Pclass
female  1         35.0
        2         28.0
        3         21.5
male    1         40.0
        2         30.0
        3         25.0
Name: Age, dtype: float64

Filled Embarked missing values with most common: 'S'


## 4. Feature Engineering (Creating New Data)

Feature engineering is the process of creating new columns (features) from existing ones to give the model better predictive power.

Common techniques include:
* **Combining:** `SibSp` + `Parch` to get `FamilySize`.
* **Extracting:** Getting the `Title` from the `Name` string.
* **Transforming:** Categorizing numerical data (e.g., `Age` to `AgeGroup`).

In [9]:
df_features = df_filled.copy()

# Feature 1: Age Group (Transformation)
def categorize_age(age):
    if pd.isnull(age):
        return 'Unknown'
    elif age < 12:
        return 'Child'
    elif age < 65:
        return 'Adult'
    else:
        return 'Elder'

df_features['AgeGroup'] = df_features['Age_filled_smart'].apply(categorize_age)
print("Age group distribution:")
print(df_features['AgeGroup'].value_counts())

# Feature 2 & 3: Family Size and Is Alone (Combining and Binary)
df_features['FamilySize'] = df_features['SibSp'] + df_features['Parch'] + 1
df_features['IsAlone'] = (df_features['FamilySize'] == 1).astype(int)
print(f"\nPassengers traveling alone: {df_features['IsAlone'].sum()}")

# Feature 4: Title (Extraction)
df_features['Title'] = df_features['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
title_mapping = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master'
}
df_features['Title_simple'] = df_features['Title'].map(title_mapping).fillna('Other')
print("Simplified title distribution:")
print(df_features['Title_simple'].value_counts())

# Feature 5: Deck (Extraction and Categorical)
df_features['Deck'] = df_features['Cabin'].str[0].fillna('Unknown')
print(f"\nDeck distribution:")
print(df_features['Deck'].value_counts())

Age group distribution:
AgeGroup
Adult    812
Child     68
Elder     11
Name: count, dtype: int64

Passengers traveling alone: 537
Simplified title distribution:
Title_simple
Mr        517
Miss      182
Mrs       125
Master     40
Other      27
Name: count, dtype: int64

Deck distribution:
Deck
Unknown    687
C           59
B           47
D           33
E           32
A           15
F           13
G            4
T            1
Name: count, dtype: int64


In [10]:
# Summary Printout
datasets = {
    'Original': titanic_df,
    'Filled_Smart': df_filled,
    'With_Features': df_features
}

print(f"\n=== CLEANING IMPACT ON SIZE ===")
for name, df in datasets.items():
    print(f"  {name}: {len(df)} rows, {len(df.columns)} columns")

print(f"\n=== IMPACT ON KEY INSIGHTS (Gender Survival) ===")
for name, df in datasets.items():
    if 'Survived' in df.columns and 'Sex' in df.columns:
        gender_survival = df.groupby('Sex')['Survived'].mean()
        print(f"{name} Data:")
        for gender, rate in gender_survival.items():
            print(f"  {gender.capitalize()} survival: {rate:.1%}")
        print()


# Export cleaned datasets
os.makedirs('data-visualization/data', exist_ok=True)

# 1. Minimal Clean (for comparison)
minimal_clean = titanic_df.dropna(subset=['Survived', 'Sex', 'Pclass'])
minimal_clean.to_csv('data-visualization/data/titanic_minimal_clean.csv', index=False)

# 2. Smart Filled Data
smart_clean_cols = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex',
                   'Age_filled_smart', 'SibSp', 'Parch', 'Fare', 'Embarked_filled']
smart_clean = df_filled[smart_clean_cols].copy()
smart_clean.columns = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex',
                       'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
smart_clean.to_csv('data-visualization/data/titanic_smart_filled.csv', index=False)

# 3. Featured Data (The most ready-to-model set)
feature_columns = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age_filled_smart',
                   'SibSp', 'Parch', 'Fare', 'Embarked_filled',
                   'AgeGroup', 'FamilySize', 'IsAlone', 'Title_simple', 'Deck']
featured_data = df_features[feature_columns].copy()
featured_data = featured_data.rename(columns={'Age_filled_smart': 'Age', 'Embarked_filled': 'Embarked'})
featured_data.to_csv('data-visualization/data/titanic_with_features.csv', index=False)

print(f"\n📁 Exported datasets:")
print(f"- titanic_minimal_clean.csv: {len(minimal_clean)} rows, basic cleaning")
print(f"- titanic_smart_filled.csv: {len(smart_clean)} rows, advanced filling")
print(f"- titanic_with_features.csv: {len(featured_data)} rows, {len(featured_data.columns)} columns")

print(f"\n🎯 KEY LEARNING:")
print("Strategic cleaning and feature engineering are essential steps that directly impact model performance and analytical conclusions!")


=== CLEANING IMPACT ON SIZE ===
  Original: 891 rows, 12 columns
  Filled_Smart: 891 rows, 16 columns
  With_Features: 891 rows, 22 columns

=== IMPACT ON KEY INSIGHTS (Gender Survival) ===
Original Data:
  Female survival: 74.2%
  Male survival: 18.9%

Filled_Smart Data:
  Female survival: 74.2%
  Male survival: 18.9%

With_Features Data:
  Female survival: 74.2%
  Male survival: 18.9%


📁 Exported datasets:
- titanic_minimal_clean.csv: 891 rows, basic cleaning
- titanic_smart_filled.csv: 891 rows, advanced filling
- titanic_with_features.csv: 891 rows, 15 columns

🎯 KEY LEARNING:
Strategic cleaning and feature engineering are essential steps that directly impact model performance and analytical conclusions!
